In [11]:
import warnings
from math import comb

import numpy as np
import pandas as pd
from scipy.stats import bernoulli, logistic

warnings.filterwarnings("ignore")


In [12]:
# Pooling====
# GROUP TESTING
# CREATED: Eli P. Fenichel, Yale University
# LAST UPDATED: July 15, 2020
# Probability of a groups testing positive accounting for sensitivity of test
# with prevalence m, group size = group, population = pop


def estimate_pooling_prob_of_pos_groups(group_size, m, pop, se, s_loss):
    shat = se - group_size * s_loss
    p_pos = 1 - (1 - shat * m) ** group_size  # shat es la sensibilidad,
    # p.pos: Probabilidad que un grupo sea positivo
    # (1-prevalencia): Probabilidad que se salga negativo una persona
    # (1-prevalencia)^tamano_grupo: Probabilidad que todos sean negativos en el grupo
    return p_pos


In [13]:
# Group results
def estimate_pooling_tests(group_size, p, pop, se, s_loss, sp):
    p_pos_pool = estimate_pooling_prob_of_pos_groups(
        group_size, p, pop, se, s_loss)
    p_pos_test = se * p + (1 - sp) * (1 - p)
    # number of groups
    n_group = int(np.round(pop / group_size))
    pos_groups = np.sum(bernoulli.rvs(p_pos_pool, size=n_group))
    neg_groups = n_group - pos_groups  # cambio

    # number of people
    neg_peop_group = neg_groups * group_size
    pos_peop_group = pos_groups * group_size

    pos_peop = pos_peop_group * p_pos_test

    n_groups_pooling = pos_groups + neg_groups
    total_tests_pooling = n_groups_pooling + pos_peop
    test_per_person_pooling = total_tests_pooling / pop

    # the expected number of people in negative groups that are actually
    # false positives with group size = group.size, prevalence p, sensitivity s,
    # in a population p.

    shat = se - group_size * s_loss
    t_prob = (1 - shat) * p + (1 - p)
    expected_number_missed = np.zeros(group_size - 1)
    for x in range(group_size - 1):
        expected_number_missed[x] = (
            comb(group_size, group_size - (x + 1))
            * ((1 - p) / t_prob) ** (group_size - (x + 1))
            * ((1 - shat) * p / t_prob) ** (x + 1)
        )

    exp_m = np.matmul(range(1, group_size), expected_number_missed)
    gm = exp_m * neg_groups

    res = pd.DataFrame(
        {
            "p_pos_pool": [p_pos_pool],
            "p_pos_test": [p_pos_test],
            "pos_groups": [pos_groups],
            "neg_groups": [neg_groups],
            "pos_peop_group": [pos_peop_group],
            "pos_peop": [pos_peop],
            "neg_peop_group": [neg_peop_group],
            "n_groups_pooling": [n_groups_pooling],
            "n_undetected_pos": [gm],
            "undetected_groups": [exp_m],
            "total_tests_pooling": [total_tests_pooling],
            "number_test_per_person_pooling": [test_per_person_pooling],
        }
    )
    return res


In [14]:
def covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    sens_ag_test,
    spec_ag_test,
    sens_pcr_test,
    spec_pcr_test,
    sens_lamp_test,
    spec_lamp_test,
    people,
    cost_pcr,
    cost_antigen,
    cost_lamp,
    group_size,
    strategy,
    budget,
):
    df_out = pd.DataFrame({})
    sens = sens_model
    spec = spec_model
    # for sens in sens_model:
    #     for spec in spec_model:

    # Probability that an a test gives a positive result
    # Antigen
    prob_res_pos_ag = sens_ag_test * prevalence + \
        (1 - spec_ag_test) * (1 - prevalence)
    # LAMP
    prob_res_pos_lamp = sens_lamp_test * prevalence + (1 - spec_lamp_test) * (
        1 - prevalence
    )
    # PCR
    prob_res_pos_pcr = sens_pcr_test * prevalence + (1 - spec_pcr_test) * (
        1 - prevalence
    )
    # Probability that a test gives a negative result
    prob_res_neg_ag = 1 - prob_res_pos_ag
    prob_res_neg_lamp = 1 - prob_res_pos_lamp
    prob_res_neg_pcr = 1 - prob_res_pos_pcr

    # Probability model gives a low risk classification
    # prob_model_low_risk = (1 - sens) * prevalence +
    #   spec * (1 - prevalence)
    # prob_model_high_risk = 1 - prob_model_low_risk

    prob_model_high_risk_ag = sens * \
        prob_res_pos_ag + (1 - spec) * prob_res_neg_ag
    # prob_model_high_risk_ag = 1 - prob_model_low_risk_ag

    prob_model_high_risk_pcr = sens * \
        prob_res_pos_pcr + (1 - spec) * prob_res_neg_pcr
    # prob_model_high_risk_pcr = 1 - prob_model_low_risk_pcr

    q_model_high_risk_ag = logistic.ppf(prob_model_high_risk_ag)
    q_model_high_risk_pcr = logistic.ppf(prob_model_high_risk_pcr)

    prob_model_high_risk = logistic.cdf(
        0.5 * (q_model_high_risk_ag + q_model_high_risk_pcr)
    )
    prob_model_low_risk = 1 - prob_model_high_risk

    # PPV y NPV of model
    npv_model_ag = spec * prob_res_neg_ag / prob_model_low_risk
    npv_model_pcr = spec * prob_res_neg_pcr / prob_model_low_risk

    q_npv_model_low_risk_ag = logistic.ppf(npv_model_ag)
    q_npv_model_low_risk_pcr = logistic.ppf(npv_model_pcr)

    npv_model = logistic.cdf(
        0.5 * (q_npv_model_low_risk_ag + q_npv_model_low_risk_pcr))
    #
    # npv_model <- spec * (1 - prevalence) /
    #   ((1 - sens) * prevalence + spec * (1 - prevalence))

    # People in risk
    people_high_risk = np.sum(bernoulli.rvs(prob_model_high_risk, size=people))
    people_low_risk = people - people_high_risk

    # Probability that given High risk classification
    # the patient has a positive result with antigen

    prob_res_pos_given_high_risk_ag = sens * prob_res_pos_ag / prob_model_high_risk

    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    prob_res_pos_given_low_risk_ag = (
        1 - sens) * prob_res_pos_ag / prob_model_low_risk

    prob_res_pos_given_high_risk_lamp = sens * \
        prob_res_pos_lamp / prob_model_high_risk

    prob_res_neg_given_high_risk_lamp = 1 - prob_res_pos_given_high_risk_lamp

    prob_res_pos_given_high_risk_pcr = sens * \
        prob_res_pos_pcr / prob_model_high_risk

    prob_res_neg_given_high_risk_pcr = 1 - prob_res_pos_given_high_risk_pcr

    # Probability that given High risk classification
    # the patient has a negative result with antigen
    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    # PPV golden test
    ppv_golden_test = (
        sens_pcr_test
        * prevalence
        / (sens_pcr_test * prevalence + (1 - spec_pcr_test * (1 - prevalence)))
    )

    # Strategy 1 -------------------------------------------------------
    if strategy == 1:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            # Probability of people tested before of 5 days since symptoms onset.
            prob_symp_less_5d = 1 - prob_symp_more_5d

            #             people_low_risk = 0

            #             # Number of antigen tests done
            #             number_alt_tests = np.sum(
            #                 bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            #             )

            #             # Number of PCR tests done
            #             number_pcr_tests = (
            #                 people_high_risk
            #                 - number_alt_tests
            #                 + np.sum(
            #                     bernoulli.rvs(
            #                         prob_res_neg_given_high_risk_ag, size=number_alt_tests
            #                     )
            #                 )
            #             )

            #             number_test_per_person = (number_pcr_tests + number_alt_tests) / (
            #                 people_high_risk
            #             )

            #             number_positive_reported = (
            #                 people_res_pos_given_high_risk_ag
            #                 + people_res_pos_given_res_neg_ag_high_risk
            #                 + people_res_pos_given_high_risk_pcr
            #                 + pooling_tests_low_risk["pos_peop"]
            #             )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                              size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag,
                              size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (
                people_high_risk
            )
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
            )

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": [total_cost_alt],
                    "total_cost_pcr": [total_cost_pcr],
                    "total_cost": [total_cost],
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 2:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d

            pooling_tests_full_prevalence = pd.DataFrame()

            pooling_tests_full_prevalence = estimate_pooling_tests(
                group_size=group_size,
                p=prevalence,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            pooling_tests_low_risk = estimate_pooling_tests(
                group_size=group_size,
                p=1 - npv_model,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                              size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag,
                              size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
                + pooling_tests_low_risk["total_tests_pooling"].values
            )
            number_test_per_person = (
                number_pcr_tests + number_alt_tests) / (people)
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + pooling_tests_low_risk["pos_peop"]
            )

            # number_positive_reported <-
            #   people_high_risk * prob_symp_less_5d *
            #   (
            #     prob_res_pos_given_high_risk_ag +
            #       prob_res_neg_given_high_risk_ag *
            #         prob_res_pos_given_high_risk_pcr
            #   ) +
            #   people_high_risk * prob_symp_more_5d *
            #     prob_res_pos_given_high_risk_pcr +
            #   pooling_tests_low_risk$pos_peop
            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 3:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                              size=people_low_risk)
            )

            number_alt_tests = (
                people_symp_less_5d + people_low_risk + people_res_pos_given_low_risk_ag
            )

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag,
                              size=people_symp_less_5d)
            )

            number_pcr_tests = people_symp_more_5d + people_res_neg_given_high_risk_ag

            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            people_confirmed_retest_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag **
                              2, size=people_low_risk)
            )

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + people_confirmed_retest_ag
            )

            number_test_per_person = (
                number_pcr_tests + number_alt_tests) / (people)
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 4:
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            prob_model_first_responder_pcr = 0.01
            people_first_responder = np.sum(
                bernoulli.rvs(prob_model_first_responder_pcr, size=people)
            )
            people_high_risk = np.sum(
                bernoulli.rvs(
                    prob_model_high_risk, size=people - people_first_responder
                )
            )
            people_low_risk = people - people_first_responder - people_high_risk

            # Number of test
            number_pcr_tests = people_first_responder

            number_lamp_tests = (
                np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_lamp, size=people_high_risk
                    )
                )
                + people_high_risk
            )

            number_ag_tests = (
                np.sum(
                    bernoulli.rvs(prob_res_pos_given_low_risk_ag,
                                  size=people_low_risk)
                )
                + people_low_risk
            )

            number_alt_tests = number_lamp_tests + number_ag_tests

            Highrisk = np.sum(
                bernoulli.rvs(prob_res_pos_given_high_risk_pcr,
                              size=people_high_risk)
            )

            number_positive_reported = (
                np.sum(bernoulli.rvs(prob_res_pos_pcr,
                       size=people_first_responder))
                + Highrisk
                + np.sum(
                    bernoulli.rvs(
                        1 - prob_res_pos_given_high_risk_pcr, size=Highrisk)
                )
                + np.sum(
                    bernoulli.rvs(
                        prob_res_pos_given_low_risk_ag**2, size=people_low_risk
                    )
                )
            )

            number_test_per_person = (
                number_pcr_tests + number_alt_tests) / (people)

            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost_alt = (
                cost_lamp * number_lamp_tests + cost_antigen * number_ag_tests
            )
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / total_cost) * (
                (people_high_risk + people_low_risk +
                 people_first_responder) / people
            )

            efficiency = (number_positive_reported / total_cost) * (
                (people_high_risk + people_low_risk +
                 people_first_responder) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [
                        people_high_risk + people_low_risk + people_first_responder
                    ],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "people_first_responder": [people_first_responder],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "sens_lamp_test": [sens_lamp_test],
                    "spec_lamp_test": [spec_lamp_test],
                    "prob_model_first_responder_pcr,": [prob_model_first_responder_pcr],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )
            df_out = pd.concat([df_out, df2])

    return df_out


In [15]:
def r_covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    strategy,
    people,
    n_simul=1000,
    sens_ag_test=0.8,
    spec_ag_test=0.95,
    sens_pcr_test=0.95,
    spec_pcr_test=0.95,
    sens_lamp_test=0.95,
    spec_lamp_test=0.95,
    cost_pcr=100,
    cost_antigen=50,
    cost_lamp=20,
    group_size=5,
    budget=100000,
):
    resultados = pd.DataFrame()
    for s in range(n_simul):
        df = covid_testing_strategies(
            prevalence,
            sens_model,
            spec_model,
            sens_ag_test,
            spec_ag_test,
            sens_pcr_test,
            spec_pcr_test,
            sens_lamp_test,
            spec_lamp_test,
            people,
            cost_pcr,
            cost_antigen,
            cost_lamp,
            group_size,
            strategy,
            budget,
        )
        df["nsimul"] = s + 1
        resultados = pd.concat([resultados, df], ignore_index=True)
    return resultados


In [16]:
model_parameters = pd.read_pickle(
    filepath_or_buffer="data/model_parameters.pkl")


In [17]:
df_strategy_1 = r_covid_testing_strategies(
    strategy=1,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_1.to_pickle(path="data/df_strategy_1.pkl")
df_strategy_1.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,1,0.269096,1000,381,619,0.796296,0.761092,0.8,0.95,0.75,...,235,292,269.096355,1.377953,14500,23500,38000,1381.578947,768.421053,1
1,1,0.269096,1000,381,619,0.796296,0.761092,0.8,0.95,0.50,...,298,261,269.096355,1.249344,8900,29800,38700,1229.974160,674.418605,1
2,1,0.269096,1000,381,619,0.796296,0.761092,0.8,0.95,0.25,...,323,246,269.096355,1.123360,5250,32300,37550,1139.813582,655.126498,1
3,1,0.269096,1000,383,617,0.796296,0.761092,0.8,0.95,0.75,...,222,282,269.096355,1.328982,14350,22200,36550,1392.612859,771.545828,2
4,1,0.269096,1000,383,617,0.796296,0.761092,0.8,0.95,0.50,...,277,266,269.096355,1.227154,9650,27700,37350,1258.366801,712.182062,2
5,1,0.269096,1000,383,617,0.796296,0.761092,0.8,0.95,0.25,...,332,254,269.096355,1.112272,4700,33200,37900,1124.010554,670.184697,2
6,1,0.269096,1000,400,600,0.796296,0.761092,0.8,0.95,0.75,...,227,314,269.096355,1.322500,15100,22700,37800,1399.470899,830.687831,3
7,1,0.269096,1000,400,600,0.796296,0.761092,0.8,0.95,0.50,...,284,284,269.096355,1.215000,10100,28400,38500,1262.337662,737.662338,3
8,1,0.269096,1000,400,600,0.796296,0.761092,0.8,0.95,0.25,...,342,256,269.096355,1.102500,4950,34200,39150,1126.436782,653.895275,3
9,1,0.269096,1000,373,627,0.796296,0.761092,0.8,0.95,0.75,...,233,278,269.096355,1.327078,13100,23300,36400,1359.890110,763.736264,4


In [18]:
df_strategy_2 = r_covid_testing_strategies(
    strategy=2,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_2.to_pickle(path="data/df_strategy_2.pkl")
df_strategy_2.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,2,0.269096,1000,414,586,0.796296,0.761092,0.8,0.95,0.75,...,391.575628,350.575628,269.096355,0.702576,15550,39157.562836,54707.562836,1284.238580,640.817485,1
1,2,0.269096,1000,414,586,0.796296,0.761092,0.8,0.95,0.50,...,440.025808,319.025808,269.096355,0.654026,10700,44002.580789,54702.580789,1195.603203,583.200652,1
2,2,0.269096,1000,414,586,0.796296,0.761092,0.8,0.95,0.25,...,510.800718,290.800718,269.096355,0.610801,5000,51080.071813,56080.071813,1089.158231,518.545552,1
3,2,0.269096,1000,390,610,0.796296,0.761092,0.8,0.95,0.75,...,374.315844,328.315844,269.096355,0.675316,15050,37431.584380,52481.584380,1286.767257,625.582950,2
4,2,0.269096,1000,390,610,0.796296,0.761092,0.8,0.95,0.50,...,418.315844,296.315844,269.096355,0.624316,10300,41831.584380,52131.584380,1197.576961,568.399843,2
5,2,0.269096,1000,390,610,0.796296,0.761092,0.8,0.95,0.25,...,472.060503,248.060503,269.096355,0.580061,5400,47206.050269,52606.050269,1102.649790,471.543675,2
6,2,0.269096,1000,399,601,0.796296,0.761092,0.8,0.95,0.75,...,397.540934,324.540934,269.096355,0.696541,14950,39754.093356,54704.093356,1273.288507,593.266269,3
7,2,0.269096,1000,399,601,0.796296,0.761092,0.8,0.95,0.50,...,428.155700,303.155700,269.096355,0.636156,10400,42815.570017,53215.570017,1195.431525,569.674815,3
8,2,0.269096,1000,399,601,0.796296,0.761092,0.8,0.95,0.25,...,487.960862,298.960862,269.096355,0.588961,5050,48796.086175,53846.086175,1093.785832,555.213727,3
9,2,0.269096,1000,411,589,0.796296,0.761092,0.8,0.95,0.75,...,406.315844,344.315844,269.096355,0.698316,14600,40631.584380,55231.584380,1264.341502,623.403887,4


In [19]:
df_strategy_3 = r_covid_testing_strategies(
    strategy=3,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_3.to_pickle(path="data/df_strategy_3.pkl")
df_strategy_3.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,3,0.269096,1000,379,621,0.796296,0.761092,0.8,0.95,0.75,...,228,280,269.096355,1.202,48700,22800,71500,1681.118881,391.608392,1
1,3,0.269096,1000,379,621,0.796296,0.761092,0.8,0.95,0.50,...,282,273,269.096355,1.152,43500,28200,71700,1606.694561,380.753138,1
2,3,0.269096,1000,379,621,0.796296,0.761092,0.8,0.95,0.25,...,324,242,269.096355,1.109,39250,32400,71650,1547.801814,337.752966,1
3,3,0.269096,1000,397,603,0.796296,0.761092,0.8,0.95,0.75,...,238,304,269.096355,1.211,48650,23800,72450,1671.497585,419.599724,2
4,3,0.269096,1000,397,603,0.796296,0.761092,0.8,0.95,0.50,...,296,279,269.096355,1.140,42200,29600,71800,1587.743733,388.579387,2
5,3,0.269096,1000,397,603,0.796296,0.761092,0.8,0.95,0.25,...,343,256,269.096355,1.086,37150,34300,71450,1519.944017,358.292512,2
6,3,0.269096,1000,374,626,0.796296,0.761092,0.8,0.95,0.75,...,213,301,269.096355,1.177,48200,21300,69500,1693.525180,433.093525,3
7,3,0.269096,1000,374,626,0.796296,0.761092,0.8,0.95,0.50,...,273,272,269.096355,1.146,43650,27300,70950,1615.221987,383.368569,3
8,3,0.269096,1000,374,626,0.796296,0.761092,0.8,0.95,0.25,...,325,238,269.096355,1.117,39600,32500,72100,1549.237171,330.097087,3
9,3,0.269096,1000,383,617,0.796296,0.761092,0.8,0.95,0.75,...,228,297,269.096355,1.169,47050,22800,69850,1673.586256,425.196850,4


In [20]:
df_strategy_4 = r_covid_testing_strategies(
    strategy=4,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_4.to_pickle(path="data/df_strategy_4.pkl")
df_strategy_4.head(n=12)


,strategy,prevalence,people,people_high_risk,people_low_risk,people_first_responder,sens_model,spec_model,sens_ag_test,spec_ag_test,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,4,0.269096,1000,406,580,14,0.796296,0.761092,0.8,0.95,...,14,332,269.096355,1.221,42950,1400,44350,2753.100338,748.590755,1
1,4,0.269096,1000,358,631,11,0.796296,0.761092,0.8,0.95,...,11,287,269.096355,1.194,43970,1100,45070,2649.212336,636.787220,1
2,4,0.269096,1000,402,587,11,0.796296,0.761092,0.8,0.95,...,11,345,269.096355,1.207,42730,1100,43830,2753.821583,787.132101,1
3,4,0.269096,1000,404,585,11,0.796296,0.761092,0.8,0.95,...,11,368,269.096355,1.226,43560,1100,44660,2745.185849,824.003583,2
4,4,0.269096,1000,400,590,10,0.796296,0.761092,0.8,0.95,...,10,339,269.096355,1.233,43690,1000,44690,2759.006489,758.558962,2
5,4,0.269096,1000,385,606,9,0.796296,0.761092,0.8,0.95,...,9,315,269.096355,1.206,43590,900,44490,2710.721510,708.024275,2
6,4,0.269096,1000,382,612,6,0.796296,0.761092,0.8,0.95,...,6,327,269.096355,1.192,43370,600,43970,2710.939277,743.688879,3
7,4,0.269096,1000,398,592,10,0.796296,0.761092,0.8,0.95,...,10,371,269.096355,1.211,43130,1000,44130,2744.164967,840.697938,3
8,4,0.269096,1000,394,600,6,0.796296,0.761092,0.8,0.95,...,6,358,269.096355,1.199,43390,600,43990,2725.619459,813.821323,3
9,4,0.269096,1000,389,606,5,0.796296,0.761092,0.8,0.95,...,5,325,269.096355,1.232,44370,500,44870,2745.709828,724.314687,4
